In [1]:
import pandas as pd

import string
import re

from Helper import *
from Constants import *

from collections import Counter


import nltk
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords

en_stopwords = set(stopwords.words("english"))
en_stopwords.update([s.capitalize() for s in stopwords.words("english")])

In [2]:
def preprocess(quote):
    """
    Lower first letter of the sentence
    Remove number
    Tokenize
    Remove stop words and words of len = 1
    Lemmatize
    Remove stop words and words of len = 1 (again)
    """
    
    
    # Lower first word of the sentence
    lower_first_word = lambda tab: " ".join([tab[0].lower()] + tab[1:])
    quote = " ".join([lower_first_word(sentence.split(" ")) for sentence in quote.split(".")])
    
    #Remove Numbers
    quote = re.sub(r'\d+', '', quote) 

    # Tokenize
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    word_tokens = tokenizer.tokenize(quote)
            
    remove_stop_words = lambda wt: [w for w in wt if not w in en_stopwords and len(w) > 1]
    # Remove stop words and single letters
    word_tokens = remove_stop_words(word_tokens)

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
    
    # Remove stop words and single letters
    word_tokens = remove_stop_words(word_tokens)
        
    return word_tokens

In [4]:
json_reader = pd.read_json(FILE_IN,lines=True,chunksize=10_000,compression='bz2') 

for (counter, df_chunk) in enumerate(json_reader):
    print(f"Chunk: {counter}")
    
    # Preprocess all quotes
    df_chunk["tokens"] = df_chunk["quotation"].apply(preprocess)
    
    # Transform url to newpaper acronyme
    df_chunk["newspapers"] = df_chunk["urls"].apply(lambda ln: [extract_domain(n) for n in ln])
    
    # Row = [newspaper list] [preprocessed quotation]
    df_quotes = df_chunk[["newspapers", "tokens"]]

    # Write result chunk by chunk in a csv compressed file
    add_header = (counter==0)
    write_mode = 'w' if counter == 0 else 'a'
    df_quotes.to_csv(FILE_QUOTES,header = add_header,index=False, mode=write_mode,compression='bz2')

Chunk: 0
